# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

613

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
responses = []

setlength = 49
citysets = [cities[i:i + setlength] for i in range(0, len(cities), setlength)]
citysets = citysets[0:2]

print("Beginning Data Retrieval ")
print("-----------------------------")

setcount = 1
for csets in citysets:
    
    # Loop Through List of Cities & Perform a Request for Data on Each
    recordcount = 1
    for city in csets:

        try:
            citydict = {}
            response = requests.get(f"http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&units=imperial&q={city}").json()

            citydict["city_name"] = response["name"]
            citydict["country"] = response["sys"]["country"]
            citydict["date"] = response["dt"]
            citydict["latitude"] = response["coord"]["lat"]
            citydict["longitude"] = response["coord"]["lon"]
            citydict["max_temp"] = response["main"]["temp_max"]
            citydict["humidity"] = response["main"]["humidity"]
            citydict["cloudiness"] = response["clouds"]["all"]
            citydict["wind_speed"] = response["wind"]["speed"]

            responses.append(citydict)
            print(f"Processing Record {recordcount} of Set {setcount} | {city}")

            recordcount += 1  

        except:
            print(f"{city} not found. Skipping...")

        
    setcount += 1
    time.sleep(10)

print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

Beginning Data Retrieval 
-----------------------------
City not found. Skipping...
City not found. Skipping...
Processing Record 1 of Set 1 | bethel
Processing Record 2 of Set 1 | new norfolk
Processing Record 3 of Set 1 | nikolskoye
Processing Record 4 of Set 1 | chuy
Processing Record 5 of Set 1 | rikitea
Processing Record 6 of Set 1 | morehead
Processing Record 7 of Set 1 | cape town
Processing Record 8 of Set 1 | mataura
Processing Record 9 of Set 1 | east london
Processing Record 10 of Set 1 | algiers
Processing Record 11 of Set 1 | bijar
Processing Record 12 of Set 1 | bredasdorp
City not found. Skipping...
Processing Record 13 of Set 1 | yangambi
City not found. Skipping...
Processing Record 14 of Set 1 | bluff
Processing Record 15 of Set 1 | buchanan
Processing Record 16 of Set 1 | roald
Processing Record 17 of Set 1 | san patricio
Processing Record 18 of Set 1 | norman wells
Processing Record 19 of Set 1 | yellowknife
Processing Record 20 of Set 1 | ushuaia
Processing Record 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [10]:
weatherDF = pd.DataFrame(responses)
weatherDF.count()

city_name     86
country       86
date          86
latitude      86
longitude     86
max_temp      86
humidity      86
cloudiness    86
wind_speed    86
dtype: int64

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [11]:
weatherDF.describe()

,date,latitude,longitude,max_temp,humidity,cloudiness,wind_speed
count,8.600000e+01,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000
mean,1.590067e+09,16.290581,0.196512,64.352093,65.255814,55.441860,8.222674
std,7.559842e+01,37.318488,94.485416,19.876228,24.272078,39.639435,6.454855
min,1.590067e+09,-54.800000,-175.200000,19.400000,6.000000,0.000000,0.400000
25%,1.590067e+09,-19.455000,-73.590000,50.252500,49.250000,8.750000,3.392500
50%,1.590067e+09,20.080000,4.270000,65.460000,70.000000,75.000000,6.845000
75%,1.590067e+09,42.490000,62.065000,80.277500,83.000000,90.000000,10.267500
max,1.590067e+09,77.480000,179.320000,104.250000,100.000000,100.000000,33.330000


In [ ]:
#  Get the indices of cities that have humidity over 100%.
humidityindex = weatherDF[weatherDF['humidity'] > 100].index

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
clean_city_data = weatherDF.drop(humidityindex, inplace=False)

In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv
weatherDF.to_csv("city_data.csv")

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression